In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import traceback

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("airflow_app") \
    .config('spark.executor.memory', '6g') \
    .config('spark.driver.memory', '6g') \
    .config("spark.driver.maxResultSize", "1048MB") \
    .config("spark.port.maxRetries", "100") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

your 131072x1 screen size is bogus. expect trouble
23/09/15 01:17:22 WARN Utils: Your hostname, DESKTOP-TME356J resolves to a loopback address: 127.0.1.1; using 172.29.31.176 instead (on interface eth0)
23/09/15 01:17:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/15 01:17:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/15 01:17:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [23]:
source_path = 'datalake/trusted'
write_path = 'datalake/refined'
days = 7
days_in_sec = lambda i: i * 86400
partition = 1
partition_fields = ['ano']


In [15]:
try:
    trusted_df = spark.read.format("parquet") \
        .load(source_path)
except Exception:
    print(traceback.format_exc())

23/09/15 01:31:05 INFO InMemoryFileIndex: It took 42 ms to list leaf files for 1 paths.
23/09/15 01:31:05 INFO SparkContext: Starting job: load at NativeMethodAccessorImpl.java:0
23/09/15 01:31:05 INFO DAGScheduler: Got job 4 (load at NativeMethodAccessorImpl.java:0) with 1 output partitions
23/09/15 01:31:05 INFO DAGScheduler: Final stage: ResultStage 4 (load at NativeMethodAccessorImpl.java:0)
23/09/15 01:31:05 INFO DAGScheduler: Parents of final stage: List()
23/09/15 01:31:05 INFO DAGScheduler: Missing parents: List()
23/09/15 01:31:05 INFO DAGScheduler: Submitting ResultStage 4 (MapPartitionsRDD[13] at load at NativeMethodAccessorImpl.java:0), which has no missing parents
23/09/15 01:31:05 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 103.5 KiB, free 3.4 GiB)
23/09/15 01:31:05 INFO MemoryStore: Block broadcast_6_piece0 stored as bytes in memory (estimated size 37.3 KiB, free 3.4 GiB)
23/09/15 01:31:06 INFO BlockManagerInfo: Added broadcast_6_piece0

In [8]:
trusted_df.printSchema()

root
 |-- pais: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- quantidade_confirmados: long (nullable = true)
 |-- quantidade_mortes: long (nullable = true)
 |-- quantidade_recuperados: long (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)



In [9]:
trusted_df.show(100)

23/09/15 01:22:32 INFO DataSourceStrategy: Pruning directories with: 
23/09/15 01:22:32 INFO FileSourceStrategy: Pushed Filters: 
23/09/15 01:22:32 INFO FileSourceStrategy: Post-Scan Filters: 
23/09/15 01:22:32 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 202.4 KiB, free 3.4 GiB)
23/09/15 01:22:32 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 35.4 KiB, free 3.4 GiB)
23/09/15 01:22:32 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 172.29.31.176:34579 (size: 35.4 KiB, free: 3.4 GiB)
23/09/15 01:22:32 INFO SparkContext: Created broadcast 4 from showString at NativeMethodAccessorImpl.java:0
23/09/15 01:22:32 INFO FileSourceScanExec: Planning scan with bin packing, max size: 18124764 bytes, open cost is considered as scanning 4194304 bytes.
23/09/15 01:22:32 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
23/09/15 01:22:32 INFO DAGScheduler: Got job 3 (showString at NativeMe

+-----------+------+--------+---------+-------------------+----------------------+-----------------+----------------------+----+---+
|       pais|estado|latitude|longitude|               data|quantidade_confirmados|quantidade_mortes|quantidade_recuperados| ano|mes|
+-----------+------+--------+---------+-------------------+----------------------+-----------------+----------------------+----+---+
|Afghanistan|  null|33.93911|67.709953|2021-01-01 00:00:00|                  null|             null|                 41727|2021|  1|
|Afghanistan|  null|33.93911|67.709953|2021-01-02 00:00:00|                  null|             null|                 41727|2021|  1|
|Afghanistan|  null|33.93911|67.709953|2021-01-03 00:00:00|                  null|             null|                 41727|2021|  1|
|Afghanistan|  null|33.93911|67.709953|2021-01-04 00:00:00|                  null|             null|                 42530|2021|  1|
|Afghanistan|  null|33.93911|67.709953|2021-01-05 00:00:00|          

23/09/15 01:22:32 INFO Executor: Finished task 0.0 in stage 3.0 (TID 3). 3031 bytes result sent to driver
23/09/15 01:22:32 INFO TaskSetManager: Finished task 0.0 in stage 3.0 (TID 3) in 161 ms on 172.29.31.176 (executor driver) (1/1)
23/09/15 01:22:32 INFO TaskSchedulerImpl: Removed TaskSet 3.0, whose tasks have all completed, from pool 
23/09/15 01:22:32 INFO DAGScheduler: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0) finished in 0.213 s
23/09/15 01:22:32 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
23/09/15 01:22:32 INFO TaskSchedulerImpl: Killing all running tasks in stage 3: Stage finished
23/09/15 01:22:32 INFO DAGScheduler: Job 3 finished: showString at NativeMethodAccessorImpl.java:0, took 0.219322 s


In [14]:
window_spec = (Window.partitionBy("pais").orderBy(F.col("data").cast('long')).rangeBetween(-days_in_sec(6), 0))

In [16]:
trusted_df = trusted_df.withColumn("media_movel_confirmados", F.avg("quantidade_confirmados").over(window_spec))
trusted_df = trusted_df.withColumn("media_movel_mortes", F.avg("quantidade_mortes").over(window_spec))
trusted_df = trusted_df.withColumn("media_movel_recuperados", F.avg("quantidade_recuperados").over(window_spec))

In [17]:
trusted_df.printSchema()

root
 |-- pais: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- quantidade_confirmados: long (nullable = true)
 |-- quantidade_mortes: long (nullable = true)
 |-- quantidade_recuperados: long (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- media_movel_confirmados: double (nullable = true)
 |-- media_movel_mortes: double (nullable = true)
 |-- media_movel_recuperados: double (nullable = true)



In [18]:
trusted_df.show(10)

23/09/15 01:34:39 INFO DataSourceStrategy: Pruning directories with: 
23/09/15 01:34:39 INFO FileSourceStrategy: Pushed Filters: 
23/09/15 01:34:39 INFO FileSourceStrategy: Post-Scan Filters: 
23/09/15 01:34:40 INFO CodeGenerator: Code generated in 115.739191 ms
23/09/15 01:34:40 INFO MemoryStore: Block broadcast_7 stored as values in memory (estimated size 202.4 KiB, free 3.4 GiB)
23/09/15 01:34:40 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 35.4 KiB, free 3.4 GiB)
23/09/15 01:34:40 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on 172.29.31.176:34579 (size: 35.4 KiB, free: 3.4 GiB)
23/09/15 01:34:40 INFO SparkContext: Created broadcast 7 from showString at NativeMethodAccessorImpl.java:0
23/09/15 01:34:40 INFO FileSourceScanExec: Planning scan with bin packing, max size: 18124764 bytes, open cost is considered as scanning 4194304 bytes.
23/09/15 01:34:40 INFO DAGScheduler: Registering RDD 17 (showString at NativeMethodAccessorImpl.j

+-----------+------+--------+---------+-------------------+----------------------+-----------------+----------------------+----+---+-----------------------+------------------+-----------------------+
|       pais|estado|latitude|longitude|               data|quantidade_confirmados|quantidade_mortes|quantidade_recuperados| ano|mes|media_movel_confirmados|media_movel_mortes|media_movel_recuperados|
+-----------+------+--------+---------+-------------------+----------------------+-----------------+----------------------+----+---+-----------------------+------------------+-----------------------+
|Afghanistan|  null|33.93911|67.709953|2020-01-22 00:00:00|                  null|             null|                     0|2020|  1|                    0.0|               0.0|                    0.0|
|Afghanistan|  null|33.93911|67.709953|2020-01-22 00:00:00|                  null|                0|                  null|2020|  1|                    0.0|               0.0|                    0.0|


23/09/15 01:34:48 INFO CodeGenerator: Code generated in 153.431724 ms


In [19]:
refined_df = trusted_df.select("pais",
                               "data",
                               "media_movel_confirmados",
                               "media_movel_mortes",
                               "media_movel_recuperados",
                               "ano")

In [20]:
refined_df = refined_df.withColumn("media_movel_confirmados", F.col("media_movel_confirmados").cast("long"))
refined_df = refined_df.withColumn("media_movel_mortes", F.col("media_movel_mortes").cast("long"))
refined_df = refined_df.withColumn("media_movel_recuperados", F.col("media_movel_recuperados").cast("long"))

In [21]:
refined_df.printSchema()

root
 |-- pais: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- media_movel_confirmados: long (nullable = true)
 |-- media_movel_mortes: long (nullable = true)
 |-- media_movel_recuperados: long (nullable = true)
 |-- ano: integer (nullable = true)



In [22]:
refined_df.show(10)

23/09/15 01:42:52 INFO DataSourceStrategy: Pruning directories with: 
23/09/15 01:42:52 INFO FileSourceStrategy: Pushed Filters: 
23/09/15 01:42:52 INFO FileSourceStrategy: Post-Scan Filters: 
23/09/15 01:42:52 INFO CodeGenerator: Code generated in 32.12899 ms
23/09/15 01:42:52 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 202.0 KiB, free 3.4 GiB)
23/09/15 01:42:53 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 35.2 KiB, free 3.4 GiB)
23/09/15 01:42:53 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on 172.29.31.176:34579 (size: 35.2 KiB, free: 3.4 GiB)
23/09/15 01:42:53 INFO SparkContext: Created broadcast 10 from showString at NativeMethodAccessorImpl.java:0
23/09/15 01:42:53 INFO FileSourceScanExec: Planning scan with bin packing, max size: 18124764 bytes, open cost is considered as scanning 4194304 bytes.
23/09/15 01:42:53 INFO DAGScheduler: Registering RDD 30 (showString at NativeMethodAccessorImpl

+-----------+-------------------+-----------------------+------------------+-----------------------+----+
|       pais|               data|media_movel_confirmados|media_movel_mortes|media_movel_recuperados| ano|
+-----------+-------------------+-----------------------+------------------+-----------------------+----+
|Afghanistan|2020-01-22 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-22 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-22 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-23 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-23 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-23 00:00:00|                      0|                 0|                      0|2020|
|Afghanistan|2020-01-24 00:00:00|             

23/09/15 01:42:56 INFO Executor: Finished task 0.0 in stage 10.0 (TID 14). 5705 bytes result sent to driver
23/09/15 01:42:56 INFO TaskSetManager: Finished task 0.0 in stage 10.0 (TID 14) in 2090 ms on 172.29.31.176 (executor driver) (1/1)
23/09/15 01:42:56 INFO TaskSchedulerImpl: Removed TaskSet 10.0, whose tasks have all completed, from pool 
23/09/15 01:42:56 INFO DAGScheduler: ResultStage 10 (showString at NativeMethodAccessorImpl.java:0) finished in 2.112 s
23/09/15 01:42:56 INFO DAGScheduler: Job 8 is finished. Cancelling potential speculative or zombie tasks for this job
23/09/15 01:42:56 INFO TaskSchedulerImpl: Killing all running tasks in stage 10: Stage finished
23/09/15 01:42:56 INFO DAGScheduler: Job 8 finished: showString at NativeMethodAccessorImpl.java:0, took 2.133519 s
23/09/15 01:42:56 INFO CodeGenerator: Code generated in 13.699281 ms            


23/09/15 01:42:56 INFO BlockManagerInfo: Removed broadcast_12_piece0 on 172.29.31.176:34579 in memory (size: 21.0 KiB, free: 3.4 GiB)


In [24]:
refined_df = refined_df.repartition(partition)

In [26]:
refined_df.write.mode('overwrite').format('parquet').partitionBy(partition_fields).option("parquet.compress", "snappy").save(write_path)

23/09/15 01:47:00 INFO DataSourceStrategy: Pruning directories with: 
23/09/15 01:47:00 INFO FileSourceStrategy: Pushed Filters: 
23/09/15 01:47:00 INFO FileSourceStrategy: Post-Scan Filters: 
23/09/15 01:47:01 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 202.0 KiB, free 3.4 GiB)
23/09/15 01:47:01 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 35.2 KiB, free 3.4 GiB)
23/09/15 01:47:01 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on 172.29.31.176:34579 (size: 35.2 KiB, free: 3.4 GiB)
23/09/15 01:47:01 INFO SparkContext: Created broadcast 13 from save at NativeMethodAccessorImpl.java:0
23/09/15 01:47:01 INFO FileSourceScanExec: Planning scan with bin packing, max size: 18124764 bytes, open cost is considered as scanning 4194304 bytes.
23/09/15 01:47:01 INFO DAGScheduler: Registering RDD 43 (save at NativeMethodAccessorImpl.java:0) as input to shuffle 2
23/09/15 01:47:01 INFO DAGScheduler: Got map stag

23/09/15 01:47:27 INFO BlockManagerInfo: Removed broadcast_15_piece0 on 172.29.31.176:34579 in memory (size: 20.5 KiB, free: 3.4 GiB)
23/09/15 01:47:27 INFO BlockManagerInfo: Removed broadcast_16_piece0 on 172.29.31.176:34579 in memory (size: 90.4 KiB, free: 3.4 GiB)
23/09/15 01:47:27 INFO BlockManagerInfo: Removed broadcast_13_piece0 on 172.29.31.176:34579 in memory (size: 35.2 KiB, free: 3.4 GiB)
